# SHOPRun and SHOPCase

1. Explore SHOPRun
2. Create SHOPCase
3. Trigger SHOPRun from SHOPCase
4. Edit Case File
5. Edit Case Data in Python

## Preparation

Instantiating `PowerOpsClient`...

In [ ]:
# You can control which setting files are loaded through the environmental variable below.
# In this case, the setting files are located two levels above, in the root of the repository.
import os

from cognite.powerops import PowerOpsClient

os.environ["SETTINGS_FILES"] = "../../settings.toml;../../.secrets.toml"

powerops = PowerOpsClient.from_settings()

## Explore SHOPRun

`SHOPRun` represents a single attempt at running SHOP.

In [ ]:
# Retrieve an existing shop run by event external_id.
shop_run_0 = powerops.shop.retrieve("POWEROPS_SHOP_RUN_2023-10-26T09:56:27.964670Z_efc805")
shop_run_0.check_status()

In [ ]:
print(shop_run_0)

In [ ]:
# Small helper to display first 10 lines of a long string (e.g. content of a file).
from IPython.display import HTML, display


def preview_content(content):
    N_LINES = 10
    lines = content.splitlines()
    trimmed_content = "\n".join(lines[:N_LINES])
    if len(lines) > N_LINES:
        trimmed_content += "\n..."
    display(
        HTML(
            f"<pre style='font-size: 75%; border: 1px solid var(--jp-content-font-color1); padding: 10px; margin: 10px 0 0 10px;'>{trimmed_content.strip()}</pre>"
        )
    )


# Same, just for a file.
def preview_file(file_path):
    with open(file_path) as file:
        preview_content(file.read())

In [ ]:
# Show first 10 lines of each additional SHOP file in the case.
N_LINES = 10
for shop_file_content in shop_run_0.get_shop_files():
    preview_content(shop_file_content)

# also available:
# shop_run_0.get_case_file()  # content of the case file
# list(shop_run_0.get_log_files())  # content of log files, generated by SHOP

## Create SHOPCase

`SHOPCase` is an editable representation of a SHOP case. It is used to prepare data for a SHOP run.

We run SHOP by "triggering" `SHOPCase`. This creates a new `SHOPRun` instance.

`SHOPCase` can be created from an existing `SHOPRun` instance (using `shop_run.as_case()`) or by directly instantiating the class. The former is probably useful more often.

We will create a new SHOPCase from the existing SHOPRun, then run this new case.

In [ ]:
# Create new SHOPCase from a previous SHOPRun.
case_1 = shop_run_0.to_case()
case_1

### SHOP version
There are some versions of SHOP which are already loaded in COGSHOP, the image tag `cogshop_version` in `settings.toml`. 
As of February 2024, the following SHOP versions for python version `3.9` are in the image:
* `SHOP-15.4.1.1`
* `SHOP-15.4.1.0`
* `SHOP-15.3.3.2`
* `SHOP-15.1.1.1`
* `SHOP-15.1.0.1`
* `SHOP-15.0.1.0`
* `SHOP-15.0.0.4`
* `SHOP-14.5.1.0`
* `SHOP-14.5.0.6`
* `SHOP-14.4.3.0`
* `SHOP-14.4.2.4`
* `SHOP-14.4.2.1`
* `SHOP-14.4.1.3`
* `SHOP-14.4.0.5`
* `SHOP-14.3.5.3`


In [ ]:
# See which version a case has
case_1.shop_version

In [ ]:
# See which additional versions are available remotely
powerops.shop.versions(py_version="3.9")

In [ ]:
# Changing the SHOP version for a shop run can be done like so:
case_1.shop_version = "15.3.3.2"

In [ ]:
# Run SHOP by triggering the new case.
shop_run_1 = powerops.shop.trigger_single_casefile(case_1)
shop_run_1.check_status()

In [ ]:
# A small helper function to wait for a shop run to finish.
from time import sleep


def wait(shop_run):
    while shop_run.check_status() == "in_progress":
        sleep(1)
        print(".", end="")
    print("DONE")


wait(shop_run_1)
shop_run_1.check_status()

## Edit Case File

We will save the case YAML file to local filesystem, then make some changes to it and use the changed file to run SHOP again. 

The case file is here edited using Python, but in real use it is expected that the file be edited using external tools.

In [ ]:
# Make a new SHOPCase and save the case file to local filesystem.
case = shop_run_1.to_case()
case.save_yaml("case_file.yaml")

In [ ]:
preview_file("case_file.yaml")

In [ ]:
# Deliberately break the YAML structure of the file.
# Delete "commands:" key and add a nonsense command.
with open("case_file.yaml") as case_file:
    lines = case_file.readlines()
with open("case_file.yaml", "w") as case_file:
    case_file.write("- foo\n")
    case_file.writelines(lines[17:])
preview_file("case_file.yaml")

In [ ]:
# This is now an invalid YAML file, so we cannot use it with SHOPCase.
try:
    case.load_case_file("case_file.yaml")
except Exception as exc:
    print(repr(exc))

In [ ]:
# Add "commands:" line to the beginning of the file.
# This will fix the YAML structure, but the case itself will remain broken because of the nonsense command "foo".
broken_case = open("case_file.yaml").read()
with open("case_file.yaml", "w") as case_file:
    case_file.write("commands:\n")
    case_file.write(broken_case)
preview_file("case_file.yaml")

In [ ]:
# Now we have a technically valid YAML, we can use it to run SHOP.
case.load_case_file("case_file.yaml")
shop_run_2 = powerops.shop.trigger_single_casefile(case)
shop_run_2.check_status()

In [ ]:
# We expect SHOP run to run and fail, though.
wait(shop_run_2)
shop_run_2.check_status()

In [ ]:
# More information about the failure can be obtained from the SHOP run object.
from pprint import pprint

failure_info = shop_run_2.get_failure_info()
pprint(failure_info)

In [ ]:
# it takes a few seconds for the "failures" data to become available.
while not failure_info.get("failures"):
    print(".", end="")
    sleep(1)
    failure_info = shop_run_2.get_failure_info()
print("'failures' data populated:")
pprint(failure_info)

In [ ]:
shop_run_2

## Edit Data in Python

Besides editing the case file and using `case.load_case_file()`, we can also interact with case data directly, using `case.data` dict.

We will fix the case data by adding the missing commands back.

In [ ]:
# Add missing commands back, directly into case data.
case.data["commands"] = [
    "set time_delay_unit MINUTE",
    "set ramping /on",
    "set stop_cost_from_start_cost /on",
    "set bypass_loss /on",
    "set mipgap 0.001000",
    "set timelimit 600.000",
    "set reserve_ramping_cost 1",
    "set fcr_n_equality /on",
    "set reserve_slack_cost 1",
    "set reserve_min_capacity 0.02",
    "set dyn_seg /on",
    "set dyn_juncloss /on",
    "penalty flag /on /plant /schedule",
    "start sim 3",
    "set code /inc",
    "start sim 4",
]

In [ ]:
# This time we expect the SHOP run to succeed.
shop_run_3 = powerops.shop.trigger_single_casefile(case)
wait(shop_run_3)
shop_run_3.check_status()

## Cleanup

In [ ]:
# Remove the local case file.
import os

os.unlink("case_file.yaml")

End of notebook.